## Overview
This notebook is a continuation of the simple peceptron
*  We will train the perceptron on a deck of cards. We will re-use the trained weights from the previous trainings.
    * This should test how the perceptron behaves when it sees completely new cards.
    * The errors *should* reduce as the percerptron sees more cards and the weights get better over time. 

## The Data
* Each card is generated as a 8x8 matrix. 
* At some random horizontal offset (in the card below its `y>5`), the card will only output positive values.
* Positive output is shows in green, negative in red.
* The card, in essence, is horizontally partitioned between positive and negative regions.
* The perceptron should be able to learn this boundary.

<img src="../static/h1.png" alt="Card" style="width: 200px;">

## The Deck
We generate multiple cards to create a deck of cards. They all have one common quality - they are all horizontally partitioned

<img src="../static/h1.png" alt="Card" style="width: 200px;">

<img src="../static/h2.png" alt="Card" style="width: 200px;">

### Input Vector
For training purposes, the inputs are stored as a one dimensional array X, of 64 elements. 
* Inputs are the `(x,y)` co-ordinates of the card. Hence, in a range of `(0,0)` to `(7,7)`.
* Green corresponds to inputs that must produce a positive output, red to negative. 

E.g. here the inputs will be represented as

``` python
    X =  [[0,0],[0,1],[0, 2] .. [0, 7],[1, 0],[1, 1] .. [7, 7,]]
```

### Output Vector
The outputs are correspondingly stored in a one dimensional array Y, of 64 elements.
* Every value is `1` or `-1`
* Every 8 elements represent a vertical column in the card,

![Serialized Output](../static/serialized-y.png)

``` python
   

#(x,y): (0,0)   (0,1)   (0,2)   (0,3)   (0,4)   (0,5)   (0,6)   (0,7)        
    Y = [
        -1,     -1,     -1,     -1,     -1,     -1,     1,      1, 
        
       ...
       
        -1,     -1,     -1,     -1,     -1,     -1,     1,      1,         
    ]
#(x,y): (7,0)   (7,1)   (7,2)   (7,3)   (7,4)   (7,5)   (7,6)   (7,7)    
```


In [1]:
import numpy as np
import copy
from matplotlib import pyplot as plt
import random
%matplotlib inline

In [2]:

#generate a training card with a rectangular section marked positive 
def training_card(size=8, horizontal_partition = True, vertical_partition = True):
    X = np.zeros((size*size,2))
    Y = np.zeros([X.shape[0]])
    
    s = 0
    t_y = random.uniform(size/10,9*size/10) if horizontal_partition else -1
    t_x = random.uniform(0,size) if vertical_partition else -1  
    print("card_t:",t_x,t_y)
    for i in range (0,size):    
        for j in range (0,size):
            X[s]=[i,j] 
            Y[s]= 1 if (j> t_y and i>t_x) else -1
            s+=1
    return (X,Y)            

def draw_card(X,Y):
    plt.figure(figsize=(2,2))
    for i, x in enumerate(X):
        if Y[i] > 0:
            plt.scatter(x[0], x[1], s=100, marker='s', color='green',linewidths=2)
        # Plot the positive samples
        else:
            plt.scatter(x[0], x[1], s=100, marker='s', color='red', linewidths=2)
        

#### Perceptron Algorithm

$$ 
    z = \sum_{i=1}^n x_i w_i + b 
$$

$$ 
    output = \begin{cases}
        1 & \text{if }\ z > T \text{\, where T is some threshold }
        \\-1 & \text{otherwise}
        \end{cases}
$$
#### Learning 
We adjust the weight to reduce the error
$$
    \text{if } y*a \leq 0 \text{ then }
    \bigg|\begin{multline}
    \begin{aligned}
    w_i &= w_i + y x_i \text{ for i  = 1,2,3,..,n} \\
    b &= b + y
    \end{aligned}
    \end{multline}
$$



In [3]:
import sys, site
sys.path.insert(0,"..") 
from perceptron import perceptron as tron
import importlib
importlib.reload(tron)

##
# A few debugger functions that print info out
##

# print training weights
def debug_training_weights(weights, bias, convergence):
    ##
    # Try changing the number of epochs. Lower epochs are usually better. 
    ##
    if convergence:
        print(f"CONVERGENCE! Perceptron converged in  (epoch:{convergence})")
    else: 
        print(f"Perceptron failed to converge")
    print(f"Weights => {weights}, bias={bias}\n{'-'*40}\n")

#print out any prediction errors the perceptron has
def debug_function(x,y,a):
    if (a * y) <= 0 :
        print(f"x={x}, y={y}, a={a}", "\x1b[31mMismatch\x1b[0m" )

## Generate a deck of cards
We will now generate a deck of cards, and then re-use the weights from one training to predict the next training


In [4]:
#init weights, biases to be re-used
bias = None
weights = None
num_cards = 100

#first training will predict everything wrong
prediction_errors= [1]
for c in range (0,num_cards):
    (X,Y) = training_card(vertical_partition=False)
    #draw_card(X,Y)
    #plt.show()
    
    if not bias and not weights:
        #init
        weights = np.zeros(2)
        bias = 0
    else:
        #use the prevous weights to predict this deck. How well did you do?
        p = tron.predict_only(X,Y,weights, bias)
        prediction_errors.append(p/64)
    (weights,bias, convergence) = tron.perceptron(X,Y,weights, bias, epochs=50)
    debug_training_weights(weights, bias, convergence)    


card_t: -1 1.777455921695748
CONVERGENCE! Perceptron converged in  (epoch:7)
Weights => [ 0. 13.], bias=-14.0
----------------------------------------

card_t: -1 5.276056473861462
Prediction Errors:32
CONVERGENCE! Perceptron converged in  (epoch:30)
Weights => [-1. 26.], bias=-82.0
----------------------------------------

card_t: -1 3.8419460434414594
Prediction Errors:0
CONVERGENCE! Perceptron converged in  (epoch:1)
Weights => [-1. 26.], bias=-82.0
----------------------------------------

card_t: -1 1.2541151979709526
Prediction Errors:16
CONVERGENCE! Perceptron converged in  (epoch:5)
Weights => [ 0. 39.], bias=-77.0
----------------------------------------

card_t: -1 3.563370347418809
Prediction Errors:16
CONVERGENCE! Perceptron converged in  (epoch:5)
Weights => [-1. 23.], bias=-84.0
----------------------------------------

card_t: -1 3.109958036667191
Prediction Errors:0
CONVERGENCE! Perceptron converged in  (epoch:1)
Weights => [-1. 23.], bias=-84.0
------------------------

In [5]:
#show the prediction errors
from mm_include import mermaid

def running_average(data_array):
    run_avg = []
    for i, error in enumerate(data_array):
        if i == 0:
            run_avg.append(error)
        else:
            run_avg.append((run_avg[i-1]*i + error)/(i+1))
    return run_avg    

running_avg_prediction_errors = running_average(prediction_errors)
    
mermaid(f"""
 xychart-beta
    title "Perceptron Error Rate"
    x-axis "Number of cards" {list(range(1,num_cards))}
    y-axis "Prediction Error" 0 --> 1    
    bar {prediction_errors}
    line "Prediction Error running avereg" {running_avg_prediction_errors}
    """)